#### In LangChain Thre are 2 types of Runables

1. Task Specific Runnables
   - Defination: These are core LangChain Components that have been converted into Runnables so they can be used in pipelines.
   - Purpose: Perform task-specific operation like LLM calls, prompting, reterival, etc
   - Examples:
     - `ChatOpenAI` - Runs an LLM model
     - `PromptTemplate` - Formats prompts dynamically
     - `Retriever` - Retrives relevant document

2. Runnable Primitives
    - Defination: These are fundamental building blocks for structuring  execution logic in AI workflows
    - Purpose: They help orchestrate execution by defining how different Runnables interact (sequentially, in parallel, conditionally, etc)
    - Examples:
        - `RunnableSequence`: Runs steps in **Order** (`|` operator)
        - `RunnableParallel`: Runs multiple steps **simultaneously**.
        - `RunnableMap`: Maps the same input across multiple functions.
        - `RunnableBranch`: Implements **conditional exevution** (if-else logic).
        - `RunnableLambda`: Wraps **custom Python functions** into Runnables.
        - `RunnablePassthrough`: Just forwards input as output (acts as a placeholder)

# <center>Runnable Primitives</center>

# 1. **Runnable Sequence**:


- It is a sequential chain of runnables in LangChain that executes each step one after another, passing the output of one step as the input to the next.
  
- It is useful when you need to compose multiple runnables together in a structured workflow


### Code:

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from dotenv import load_dotenv
import os


In [8]:
load_dotenv()

True

In [16]:
OPENAI_API_KEY = os.getenv('OPEN_AI_KEY')

In [33]:
prompt1 = PromptTemplate(
    template = "Write a joke about {topic}",
    input_variables=['topic'],
)

In [34]:
prompt2 = PromptTemplate(
    template="Explain the Following Joke- {joke}",
    input_variables=['joke'],
)

In [27]:
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1",

)

In [28]:
parser = StrOutputParser()

In [29]:
parser

StrOutputParser()

In [38]:
chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)

In [39]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Write a joke about {topic}')
| ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000019777CAC290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000197760A2310>, root_client=<openai.OpenAI object at 0x0000019776565DD0>, root_async_client=<openai.AsyncOpenAI object at 0x0000019777CAFCD0>, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')
| StrOutputParser()
| P

In [40]:
chain.invoke({"topic": "AI"})

'This joke plays on the word "bytes," which is a unit of digital information storage in computers, and the word "bites," which can refer to emotional baggage or issues. The joke suggests that the robot went to therapy because it had too much emotional baggage, but instead of using the term "bites," it uses "bytes" to link it to the context of a robot. Overall, it\'s a play on words that combines technology and emotional themes for a humorous outcome.'

# 2. Runnable Parallel

- **Runnable Parallel** is a runnable primitive that allows multiple runnables to execute in parallel

- Each runnable receives the same input and processes it independently, producing a dictionary of outputs.

  

### Code

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel

In [42]:
load_dotenv()

True

In [63]:
OPENAI_API_KEY = os.getenv('OPEN_AI_KEY')

In [64]:
prompt1 = PromptTemplate(
    template="Generate a tweet about {tweet_topic}",
    input_variables=['tweet_topic'],
)

In [66]:
prompt1

PromptTemplate(input_variables=['tweet_topic'], input_types={}, partial_variables={}, template='Generate a tweet about {tweet_topic}')

In [67]:
prompt2 = PromptTemplate(
    template="Generate a Linkedin Post About {linkedin_topic}",
    input_variables=['linkedin_topic']
)

In [68]:
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1",    
)

In [69]:
parser = StrOutputParser()

In [71]:
# runnable parallel process in dictk
parallel_chain = RunnableParallel(
    {
        "tweet": RunnableSequence(prompt1, model, parser), # task-1
        "linkedin": RunnableSequence(prompt2, model, parser), # task-2
    }
    # both are running in parallel
)

In [72]:
parallel_chain

{
  tweet: PromptTemplate(input_variables=['tweet_topic'], input_types={}, partial_variables={}, template='Generate a tweet about {tweet_topic}')
         | ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000197783FB350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019778332310>, root_client=<openai.OpenAI object at 0x00000197783E17D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000197783E2610>, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter

In [78]:
result = parallel_chain.invoke(
    {
        "tweet_topic": "AI", 
        "linkedin_topic":"Software Development",
    }
    # can also be use single common input
)

In [75]:
result

{'tweet': '"AI continues to transform industries with its ability to analyze vast amounts of data, automate tasks, and make insightful predictions. The future of technology is looking brighter with AI leading the way. #ArtificialIntelligence #TechRevolution"',
 'linkedin': 'Excited to be part of the software development team at [Company Name]! We are constantly innovating and pushing the boundaries of technology to create cutting-edge solutions for our clients. Looking forward to collaborating with my talented colleagues to bring our ideas to life. #softwaredevelopment #innovation #technology #codinglife'}

In [76]:
result['tweet']

'"AI continues to transform industries with its ability to analyze vast amounts of data, automate tasks, and make insightful predictions. The future of technology is looking brighter with AI leading the way. #ArtificialIntelligence #TechRevolution"'

In [77]:
result['linkedin']

'Excited to be part of the software development team at [Company Name]! We are constantly innovating and pushing the boundaries of technology to create cutting-edge solutions for our clients. Looking forward to collaborating with my talented colleagues to bring our ideas to life. #softwaredevelopment #innovation #technology #codinglife'

# 3. RunnablePassthrough

- RunnablePassthrough is a special runnable primitive that simply returns the input as output without modifying it.

### Code

we use `RunnablePassthrought` to print internmediate output
- like
    - prompt -> joke -> explanation
        - here passthrough can be used to print joke

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")

<pre>
                            |--  RunnnablePassthrough
prompt1 > model > parser -> |
                            |--  prompt2 > model > parser

In [5]:
prompt1 = PromptTemplate(
    template="Write me Joke on Topic: {joke}",
    input_variables=['joke'],
)

In [18]:
prompt2 = PromptTemplate(
    template="Write me exaplanation of this joke: {explanation_joke}",
    input_variables=['explanation_joke']
)

In [21]:
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

In [22]:
parser = StrOutputParser()

In [23]:
joke_gen_chain = RunnableSequence(
    prompt1, model, parser
)

In [24]:
print_n_explanation_chain = RunnableParallel(
    {
        "joke": RunnablePassthrough(),
        "explanation_joke": RunnableSequence(prompt2, model, parser),
    }
)

In [25]:
final_chain = RunnableSequence(joke_gen_chain, print_n_explanation_chain)

In [26]:
result = final_chain.invoke("Python")

In [27]:
result

{'joke': 'Why did the python programmer go broke?\n\nBecause he spent all his money on "Pythons" instead of actual snakes!',
 'explanation_joke': 'This joke is a play on words involving the programming language Python and actual snakes. In the context of programming, a Python programmer is someone who specializes in writing code using the Python programming language. However, the joke suggests that the Python programmer went broke because they spent all their money on "Pythons," which could be interpreted as expensive snakes, rather than actual snakes. This humorous twist on the word "Python" highlights the importance of being mindful of one\'s spending habits and making sure to prioritize financial responsibilities.'}

In [28]:
result['joke']

'Why did the python programmer go broke?\n\nBecause he spent all his money on "Pythons" instead of actual snakes!'

In [29]:
result['explanation_joke']

'This joke is a play on words involving the programming language Python and actual snakes. In the context of programming, a Python programmer is someone who specializes in writing code using the Python programming language. However, the joke suggests that the Python programmer went broke because they spent all their money on "Pythons," which could be interpreted as expensive snakes, rather than actual snakes. This humorous twist on the word "Python" highlights the importance of being mindful of one\'s spending habits and making sure to prioritize financial responsibilities.'

# 4. RunnableLambda

- RunnableLambda is a runnable primitive that allows you to apply custome Python functions within an AI pipeline

- It acts as a middleware between different AI components, enabling preprocessing, transformation, API calls, filtering and post-processing in a LangChain workflow.

### Code

In [30]:
from langchain_core.runnables import RunnableLambda

In [31]:
def word_counter(text):
    return len(text.split())

In [33]:
seq = RunnableLambda(word_counter)

In [39]:
res = seq.invoke("This is demo runnable working sample, to show text len")

In [40]:
res

10

In [41]:
# main code

In [47]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel, RunnablePassthrough

In [46]:
load_dotenv()

True

In [52]:
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")

In [50]:
prompt = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables=['topic']
)

In [55]:
model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENAI_API_KEY,
    verbose=True,
)

In [56]:
parser = StrOutputParser()

In [57]:
joke_gen_chain = RunnableSequence(prompt, model, parser)

In [59]:
parallel_chain = RunnableParallel(
    {
        "joke": RunnablePassthrough(),
        "word_count": RunnableLambda(word_counter)
    }
)

In [60]:
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

In [61]:
result = final_chain.invoke({"topic": "Python"})

In [62]:
result

{'joke': "Why was the Python programmer so stressed out? Because he couldn't find his anaconda in the code!",
 'word_count': 17}

# 5. RunnableBranch

- Runnable Branch is a control flow component in LangChain that allows you yo conditionally route input data to different chains or runnables based on custom logic.

- It functions like an if/elif/else block for chains - where you define a set of condition functions, each associated with a runnable (e.g., LLM call, prompt chain, or tool). The first matching condition is executed. If no condition matches, a default runnable is used (if provided)

### branch runnable syntax
<pre>
branch_chain = RunnableBranch(
    (condition, runnable),
    .
    .
    .
    (condition, runnable),
    default
)
</pre>

### Code

In [76]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
from langchain_core.runnables import RunnableSequence, RunnableBranch

In [94]:
load_dotenv()

True

In [95]:
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")

In [96]:
prompt1 = PromptTemplate(
    template = "Write a detailed Report on {topic}",
    input_variables=["topic"],
)

In [97]:
prompt2 = PromptTemplate(
    template="Summarize the following text \n {text}",
    input_variables = ["text"],
)

In [98]:
model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENAI_API_KEY,
    verbose=True,
)

In [99]:
parser = StrOutputParser()

In [100]:
report_gen_chain = RunnableSequence(prompt1, model, parser)

In [101]:
branch_chain = RunnableBranch(
    (lambda x: len(x.split()), RunnableSequence(prompt2, model, parser)),

    RunnablePassthrough(),    
)

In [102]:
final_chain = RunnableSequence(report_gen_chain, branch_chain)

In [103]:
result = final_chain.invoke("AI")

In [104]:
result

'Artificial intelligence (AI) is a rapidly growing field that aims to create intelligent machines capable of human-like cognitive functions. AI technologies are being implemented in various industries to improve efficiency and outcomes. Applications of AI include healthcare, finance, transportation, and manufacturing. AI offers benefits such as increased efficiency, data analysis, and improved decision-making, but also presents challenges related to ethics, privacy, and technical issues. Future trends in AI include the development of more sophisticated systems, integration with other technologies, and the focus on ethical principles and regulations. Overall, AI has the potential to revolutionize industries and improve quality of life, but responsible development and innovation are necessary to address challenges and fully leverage its potential.'